# Homework 2 - MapReduce

There is only one task in this homework. You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. A subset of the data is available as **enron_mails_small.csv** as part of this notebook. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, i.e. only relationships where email addresses end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

# Mehedi Shohag
## CSC 445-F

In [2]:
!pip install mrjob
!gdown --id 1sq4-zXn2Z82mdLSBBegEgsUsfqtgza-C -O mapreduce.py
!gdown --id 1It6GP8O2JqkmUtZKbYp1kpwpuwOXlLps -O enron_mails_small.csv

Downloading...
From: https://drive.google.com/uc?id=1sq4-zXn2Z82mdLSBBegEgsUsfqtgza-C
To: S:\MLAI\445\HW2\final\mapreduce.py

  0%|          | 0.00/2.66k [00:00<?, ?B/s]
100%|##########| 2.66k/2.66k [00:00<?, ?B/s]
Downloading...
From: https://drive.google.com/uc?id=1It6GP8O2JqkmUtZKbYp1kpwpuwOXlLps
To: S:\MLAI\445\HW2\final\enron_mails_small.csv

  0%|          | 0.00/857k [00:00<?, ?B/s]
100%|##########| 857k/857k [00:00<00:00, 13.6MB/s]


## Task 1 (10 pts)
You must complete the **MRFindReciprocal** class below (which is inherited from MRJob), and your code must run with the **mapreduce.py** package **mr.runJob()** as provided.

In [5]:
import csv
import mapreduce as mr
from mrjob.job import MRJob
from mrjob.step import MRStep

################################
### YOUR WORK SHOULD BE HERE ###
################################
class MRFindReciprocal(MRJob):
    
    hash_table = {}
    def mapper1(self, _, row):
        data = row.strip().split(',')
        email_from = data[1]
        email_to = data[2]
        email_from_list = email_from.split(',')
        for i in email_from_list:
            if "@enron.com" in i:
                email_from = i.split('@')[0]
                ffirst_name = email_from.split('.')[0]
                flast_name = email_from.split('.')[-1]
                email_from = ffirst_name.title() + " " + flast_name.title()
                To_list = email_to.split(';')
                for j in To_list:
                    if "@enron.com" in j:
                        email_to = j.split('@')[0]
                        tfirst_name = email_to.split('.')[0]
                        tlast_name = email_to.split('.')[-1]
                        email_to = tfirst_name.title() + " " + tlast_name.title()
                        yield ([email_from, email_to])
    
    def reducer1(self, email_from, email_to):
        yield (email_from, set(email_to))

    def mapper2(self, email_from, email_to):
        self.hash_table[email_from] = email_to
        for i in email_to:
            yield (i, email_from)

    def reducer2(self, rcvr, email_from):
        if rcvr in self.hash_table:
            email = (set(email_from), self.hash_table[rcvr])
            yield (rcvr, email)

    def mapper3(self, rcvr, email):
        sender, reciver = email
        for i in sender:
            if i in reciver:
                couple = (rcvr, i)
                first, second = sorted(couple)
                yield ('reciprocal', first + ':' + second)

    def reducer3(self, _, name):
        name = sorted(set(name))
        for i in name:
            first, second = i.split(':')
            if first != second:
                yield ('reciprocal', (i))

    def steps(self):
        return [MRStep(mapper=self.mapper1,
                       reducer=self.reducer1),
                MRStep(mapper=self.mapper2,
                       reducer=self.reducer2),
                MRStep(mapper=self.mapper3,
                       reducer=self.reducer3)]


###################################
### DO NOT EDIT BELOW THIS LINE ###
###################################
job = MRFindReciprocal(args=[])
with open('enron_mails_small.csv', 'r') as fi:
  next(fi)
  output = list(mr.runJob(enumerate(map(lambda x: x.strip(), fi)), job))

print(len(output))
output

35


[('reciprocal', 'Brenda Whitehead:Elizabeth Sager'),
 ('reciprocal', 'Carol Clair:Debra Perlingiere'),
 ('reciprocal', 'Carol Clair:Mark Taylor'),
 ('reciprocal', 'Carol Clair:Richard Sanders'),
 ('reciprocal', 'Carol Clair:Sara Shackleton'),
 ('reciprocal', 'Carol Clair:Tana Jones'),
 ('reciprocal', 'Debra Perlingiere:Kevin Ruscitti'),
 ('reciprocal', 'Drew Fossum:Susan Scott'),
 ('reciprocal', 'Elizabeth Sager:Janette Elbertson'),
 ('reciprocal', 'Elizabeth Sager:Mark Haedicke'),
 ('reciprocal', 'Elizabeth Sager:Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager:Richard Sanders'),
 ('reciprocal', 'Eric Bass:Susan Scott'),
 ('reciprocal', 'Fletcher Sturm:Greg Whalley'),
 ('reciprocal', 'Fletcher Sturm:Sally Beck'),
 ('reciprocal', 'Gerald Nemec:Susan Scott'),
 ('reciprocal', 'Grant Masson:Vince Kaminski'),
 ('reciprocal', 'Greg Whalley:Richard Sanders'),
 ('reciprocal', 'Janette Elbertson:Mark Taylor'),
 ('reciprocal', 'Janette Elbertson:Richard Sanders'),
 ('reciprocal', 'Liz Taylor:Mar

## Task 2 (5 points)
Please also convert your MR Job Class in Task 1 into a stand-alone `BDM_HW2.py` file that can be run directly with `python` similar to our Lab 3.

In [7]:
!python BDM_HW2.py enron_mails_small.csv

35
('reciprocal', 'Brenda Whitehead:Elizabeth Sager')
('reciprocal', 'Carol Clair:Debra Perlingiere')
('reciprocal', 'Carol Clair:Mark Taylor')
('reciprocal', 'Carol Clair:Richard Sanders')
('reciprocal', 'Carol Clair:Sara Shackleton')
('reciprocal', 'Carol Clair:Tana Jones')
('reciprocal', 'Debra Perlingiere:Kevin Ruscitti')
('reciprocal', 'Drew Fossum:Susan Scott')
('reciprocal', 'Elizabeth Sager:Janette Elbertson')
('reciprocal', 'Elizabeth Sager:Mark Haedicke')
('reciprocal', 'Elizabeth Sager:Mark Taylor')
('reciprocal', 'Elizabeth Sager:Richard Sanders')
('reciprocal', 'Eric Bass:Susan Scott')
('reciprocal', 'Fletcher Sturm:Greg Whalley')
('reciprocal', 'Fletcher Sturm:Sally Beck')
('reciprocal', 'Gerald Nemec:Susan Scott')
('reciprocal', 'Grant Masson:Vince Kaminski')
('reciprocal', 'Greg Whalley:Richard Sanders')
('reciprocal', 'Janette Elbertson:Mark Taylor')
('reciprocal', 'Janette Elbertson:Richard Sanders')
('reciprocal', 'Liz Taylor:Mark Haedicke')
('reciprocal', 'Mark Haed